# COVID-19-Tweet-Classification-Challenge-by-ZindiWeekendz

In [1]:
pwd

'C:\\Users\\Admin\\Desktop\\R projects\\COVID-19-Tweet-Classification-Challenge-by-ZindiWeekendz\\COVID-19-Tweet-Classification-Challenge-by-ZindiWeekendz-master'

In [2]:
# import data into jupyter
import pandas as pd
train = pd.read_csv('updated_train.csv')
test = pd.read_csv('updated_test.csv')
samplesub = pd.read_csv('updated_ss.csv')

In [3]:
# preview the data
train.head()

,ID,text,target
0,train_0,The bitcoin halving is cancelled due to,1
1,train_1,MercyOfAllah In good times wrapped in its gran...,0
2,train_2,266 Days No Digital India No Murder of e learn...,1
3,train_3,India is likely to run out of the remaining RN...,1
4,train_4,In these tough times the best way to grow is t...,0


In [4]:
samplesub.head()

,ID,target
0,test_2,0
1,test_3,0
2,test_4,0
3,test_8,0
4,test_9,0


In [5]:
print('shape: ', train.shape)

shape:  (5287, 3)


In [6]:
train.dtypes

ID        object
text      object
target     int64
dtype: object

In [7]:
!pip install nltk

In [8]:
# create a corpus (collection of clean words)
#
import re # regex
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
#
# create a corpus of the text column
corpus = []
for i in range(0, len(train)):
    review = re.sub('[^a-zA-Z]', ' ', train['text'][i]) # substitute every start of string formed data with a space
    review = review.lower() # change all formatting to lower
    review = review.split() # split strings to have a list of all strings
    ps = PorterStemmer() # function to reduce words to base/root
    # assign every string into 'word', if cannot be stemmed to base then set it as stopword and filter it out
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))] # removing english stopwords
    review = ' '.join(review)
    corpus.append(review)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
print('corpora dimension: ',len(corpus))

corpora dimension:  5287


In [10]:
# create a bag of words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1500)
x = cv.fit_transform(corpus).toarray()
y = train.target
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=30, random_state=42)

## Naive Bayes Classifier Algorithm

In [11]:
test.head()

,ID,text
0,test_2,Why is explained in the video take a look
1,test_3,Ed Davey fasting for Ramadan No contest
2,test_4,Is Doja Cat good or do you just miss Nicki Minaj
3,test_8,How Boris Johnson s cheery wounded in action p...
4,test_9,Man it s terrible Not even a reason to get on ...


In [12]:
test.shape

(1962, 2)

In [13]:
# fit a naive bayes classificaton algorithm
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nbmodel = nb.fit(x_train, y_train)
nbmodel

GaussianNB(priors=None, var_smoothing=1e-09)

In [14]:
# making the model prediction
nbpred = nbmodel.predict(x_test)
#
# get the logarithmic loss
from sklearn.metrics import log_loss
print(log_loss(y_test, nbmodel.predict_proba(x_test))) 

5.756463928020916


In [15]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, nbpred))

0.8333333333333334


In [16]:
# clean the test data
import re
import nltk
nltk.download('stopwords')
#
corpus = []
for i in range(0, len(test)):
    clean = re.sub('[^a-zA-Z]', ' ', test['text'][i])
    clean = clean.lower().split()
    ps = PorterStemmer()
    # return base words iff they are not english wods
    clean = [ps.stem(word) for word in clean if not word in set(stopwords.words('english'))] 
    clean = ''.join(clean)
    corpus.append(clean)
#    
# tokenize the corpus
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1500)
valid_test = cv.fit_transform(corpus).toarray()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
valid_test.shape

(1962, 1500)

In [20]:
# making the model prediction
nbpredtest = nbmodel.predict(valid_test)

1962

In [21]:
# predict probabilites and write to csv
nbprob = nbmodel.predict_proba(valid_test)

## CatBoost Classification Algorithm

In [25]:
from catboost import CatBoostClassifier
cat = CatBoostClassifier(loss_function='Logloss', iterations=300, depth=6, learning_rate=0.1, logging_level='Silent',
                        eval_metric='Accuracy', random_state=42)
catmodel = cat.fit(x_train, y_train)

In [26]:
# making the model prediction
catpred = catmodel.predict(x_test)
#
# get the logarithmic loss
from sklearn.metrics import log_loss
print(log_loss(y_test, catmodel.predict_proba(x_test)))

0.3850896179803919


In [27]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, catpred)) #accuracy

0.8666666666666667


In [29]:
# predict probabilites and write to csv
catprob = catmodel.predict_proba(valid_test)

## XGBoost Classifier Algorithm

In [32]:
from xgboost import XGBClassifier
xg = XGBClassifier(objective='binary:logistic', eval_metric='logloss', learning_rate=0.1, max_depth=4, seed=27,
                  n_estimators=20)
xgmodel = xg.fit(x_train, y_train)
xgmodel

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='logloss',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints=None, learning_rate=0.1, max_delta_step=0,
              max_depth=4, min_child_weight=1, missing=nan,
              monotone_constraints=None, n_estimators=20, n_jobs=0,
              num_parallel_tree=1, objective='binary:logistic', random_state=27,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27,
              subsample=1, tree_method=None, validate_parameters=False,
              verbosity=None)

In [33]:
# making the model prediction
xgpred = xgmodel.predict(x_test)
#
# get the logarithmic loss for the x_test
from sklearn.metrics import log_loss
print(log_loss(y_test, xgmodel.predict_proba(x_test))) # log loss 

0.5142312270899614


In [34]:
# predict probabilites
xgprob = xgmodel.predict_proba(valid_test)

In [35]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, xgpred)) #accuracy

0.7666666666666667


In [37]:
# write to csv file for submisson
# naivebayes
nb_df = pd.DataFrame({'Square_ID': test.ID, 'target': nbprob[:, 1]}) # Creating a submission file
nb_df.to_csv('Naivebayes.csv', index = False)
#
# catboost
cat_df = pd.DataFrame({'Square_ID': test.ID, 'target': catprob[:, 1]}) # Creating a submission file
cat_df.to_csv('CatBoost.csv', index = False)
#
# XGBoost 
xg_df = pd.DataFrame({'Square_ID': test.ID, 'target': xgprob[:, 1]}) # Creating a submission file
xg_df.to_csv('XGBoost.csv', index = False)

In [221]:
#####################################################################################################################